
# Decision Optimization Notebook - Retail SC Optimization 



This is an example of a 'traditional' notebook approach to implement an optimization model.<br>

Stand-alone/traditional in the sense:
* This notebook can be used as a solver notebook in DO4WS.
* Uses a few stand-alone re-usable functions defined at the start of the notebook. Includes only those functions it needs.
* All inputs, outputs and intermediate data DataFrames are defined globally.
* No object-oriented code.
* No re-use and sharing of code (no import from custom modules or packages).
* No use of `dse-do-utils`.

#### Before you begin


When you import this project on an IBM Cloud environment, insert a project access token at the top of this notebook as a code cell. <br>
If you do not see the cell above, Insert a project token: Click on **More -> Insert project token** in the top-right menu section and run the cell. <br>
![ws-project.mov](https://media.giphy.com/media/jSVxX2spqwWF9unYrs/giphy.gif)

## Table of Contents:
* [References](#references)
* [Readme](#Read_Me)
* [Package Management](#package_management)
* [Instantiate CPLEX](#instantiate_cplex)
* [Read Data Inputs](#read_data)
* [Create Decision Variables](#decision_variables)
* [Objective Function Definition](#objective)
* [Constraint Definition](#constraint)
* [Run CPLEX Model](#run_model)
* [Post Processing](#post)

## References<a class="anchor" id="references"></a>
* Authors | Jesse Slater & Wesley Beaver | IBM Cloud Pak Acceleration Team
* Contributor | Victor Terpstra | IBM Senior Data Scientist DSE | https://github.ibm.com/vterpstra/DO_Template_Projects_CPD25
* GitHub CPLEX Documentation | http://ibmdecisionoptimization.github.io/docplex-doc/mp/index.html

# Readme<a class="anchor" id="Read_Me"></a>

This notebook solves a simplified production planning model that is part of a larger supply chain optimization problem.

The supply-chain model we'll be building is a simplified version that only considers production facilities and warehouses (Suppliers, Manufacturers, Warehouse) and demand, i.e.:
* Multiple plants, from multiple suppliers in multiple locations
* Same line may be capable of producing a range of products, each with their own production rate and cost
* Multiple products, where different suppliers compete, i.e. are capable of producing the same product
* Multiple warehouses and different warehouse locations
* Warehouses have demand for the various products at various locations

Simplification:
* NOTE: In order to set a flexible foundation for various client environments, simplification must be made in lieu of company specific data. Any of the following simplifications can be resolved with actual or estimated historical cost/demand/route data.
* Scenario data including routes, estimated costs, current inventory has been affixed to the LSTM forecasted demand and saved to All_Routes_Table.csv
* Transportation cost is a hardcoded linear cost pending straight line point to point travel distance (Google API can be substituted to find route distance by roadmap)
* Transportation time is the summation of number of stops after the supplier. A route passing from one supplier to one manufacturer to one warehouse will be 2 days
* Demand forecast has already been generated by LSTM model (or other user defined model) and saved to data assets
* Optimization model will solve for the first demand period generated by the forecast
* Each supplier/manufacturer combination can produce product in demand
* No inventory
* User has enabled the decision optimization cartridge.

Supply Chain optimization will plan:
* Production activities: how much of which product to produce on which line for which warehouse
* Backlog: Whatever demand cannot be met will be stored into an order backlog

Such that:
* Minimal production cost
* Minimal travel cost
* Minimal backlog cost
* Meet demand
* Below or equal to supplier capacity

The input tables for this model:
* Product
* Location
* Plant
* Line
* Demand

# Package Management<a class="anchor" id="package_management"></a>
#### Use the following code block to install any missing packages

In [2]:
!pip install dd_scenario -q

In [3]:
!pip install docplex --upgrade

     |████████████████████████████████| 634 kB 31.0 MB/s eta 0:00:01
  Created wheel for docplex: filename=docplex-2.22.213-py3-none-any.whl size=696851 sha256=7c6e8b7b1f2978c52ccb3a18946cfdafd90cb3085bba4d65aed32514d3e09365
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/35/3e/11/e31bf877e1965b75dc2f3de4ec2d5c9d1680c6f803ef76ed9f
Successfully built docplex
  Attempting uninstall: docplex
    Found existing installation: docplex 2.20.204
    Uninstalling docplex-2.20.204:
      Successfully uninstalled docplex-2.20.204


#### Import packages and print environment health check

In [4]:
import pandas as pd
from IPython import get_ipython
import os
from tqdm.auto import tqdm
import numpy as np
import dd_scenario as dd
from docplex.mp.environment import Environment
from docplex.mp.model import Model
import warnings
warnings.filterwarnings('ignore')
env = Environment()
env.print_information()

* system is: Linux 64bit
* Python version 3.8.12, located at: /opt/conda/envs/Python-3.8-main/bin/python
* docplex is present, version is 2.22.213
* CPLEX library is present, version is 20.1.0.0, located at: /opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages
* pandas is present, version is 1.2.4


#### Prepare .lp and .cpo exports to save model later on

# Instantiate CPLEX<a class="anchor" id="instantiate_cplex"></a>
#### Set CPLEX characteristics

In [5]:
DO_MODEL_NAME = 'SC_DO'
SCENARIO_NAME = 'Warehouse_Demand'#'Scenario 1'
TIME_LIMIT = 30

#### Retrieve access token and instantiate an optimization model we will call shortly

In [6]:
model_builder = dd.ModelBuilder(name=DO_MODEL_NAME, projectId="pp-id")

# Creates a client...
client = dd.Client(project_id="pp-id", authorization="random_token")
mdl = Model(name="ProductionPlanning")

# Read Data Inputs<a class="anchor" id="read_data"></a>

#### Pull in the demand and forecast data
The first csv file contains every possible route (534,096) that a given product can take from supplier, manufacturer to warehouse. Note, that there are 4 suppliers, 3 manufacturers and 4 warehouses. Additionally, this file contains the travel costs (as described in distance of the route) and production capacity for each supplier product combination.

In [7]:
import pandas as pd
import requests

In [8]:
!ls -lh

total 49M
-rw-rw---- 1 wsuser wscommon 49M Nov  9 20:53 Historical_Product_Demand.csv


In [9]:
dataset_resp = requests.get("https://api.dataplatform.cloud.ibm.com/v2/gallery-assets/entries/1eddc77b3a4340d68f762625d4016160/data?accessKey=fa901d4777555e69b41b159f39014f29",
                            allow_redirects=True)

f = open('All_Routes_Table.csv', 'wb')
f.write(dataset_resp.content)
f.close()

!ls -lh

total 133M
-rw-rw---- 1 wsuser wscommon 84M Nov  9 20:54 All_Routes_Table.csv
-rw-rw---- 1 wsuser wscommon 49M Nov  9 20:53 Historical_Product_Demand.csv


In [10]:
df_data_1 = pd.read_csv("All_Routes_Table.csv")
df_data_1['Product_Code_Warehouse'] = df_data_1['Product_Code'] + '_' + df_data_1['Warehouse']
df_data_1.head()

Warehouse  Product_Code SupplierID ManufID WH1 WH2 WH3 WH4  Time_Enroute  \
0        W2  Product_0001         S1      M1  W1  W3  W4  W2             5   
1        W2  Product_0001         S1      M1  W1  W4  W3  W2             5   
2        W2  Product_0001         S1      M1  W3  W1  W4  W2             5   
3        W2  Product_0001         S1      M1  W3  W4  W1  W2             5   
4        W2  Product_0001         S1      M1  W4  W1  W3  W2             5   

  Destination  ...           f3           f4          f5  capacity  \
0          W2  ...  1941.680672  1574.236047  965.635097      58.0   
1          W2  ...  1238.977201   987.316035  378.715085      58.0   
2          W2  ...  1584.740888  1217.296263  965.635097      58.0   
3          W2  ...  1599.191579   990.590629  738.929463      58.0   
4          W2  ...   997.820877   746.159710  378.715085      58.0   

                      prodx  cost  Product_Code_Warehouse Forecast  \
0  Product_0001S1M1W1W3W4W2  0.02         Product_0001_W2      138   
1  Product_0001S1M1W1W4W3W2  0.02         Product_0001_W2      138   
2  Product_0001S1M1W3W1W4W2  0.02         Product_0001_W2      138   
3  Product_0001S1M1W3W4W1W2  0.02         Product_0001_W2      138   
4  Product_0001S1M1W4W1W3W2  0.02         Product_0001_W2      138   

                 xProd_vecm production_quantity  
0  Product_0001S1M1W1W3W4W2                 0.0  
1  Product_0001S1M1W1W4W3W2                 0.0  
2  Product_0001S1M1W3W1W4W2                 0.0  
3  Product_0001S1M1W3W4W1W2                 0.0  
4  Product_0001S1M1W4W1W3W2                 0.0  

[5 rows x 23 columns]

In [11]:
!wget https://raw.githubusercontent.com/IBMDataScience/sample-notebooks/master/Files/Data/Next_Three_Month_Forecasts.csv
!ls -lh

--2021-11-09 20:54:48--  https://raw.githubusercontent.com/IBMDataScience/sample-notebooks/master/Files/Data/Next_Three_Month_Forecasts.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79339 (77K) [text/plain]
Saving to: ‘Next_Three_Month_Forecasts.csv’

Next_Three_Month_Fo 100%[===================>]  77.48K  --.-KB/s    in 0.001s  

2021-11-09 20:54:49 (61.9 MB/s) - ‘Next_Three_Month_Forecasts.csv’ saved [79339/79339]

total 133M
-rw-rw---- 1 wsuser wscommon 84M Nov  9 20:54 All_Routes_Table.csv
-rw-rw---- 1 wsuser wscommon 49M Nov  9 20:53 Historical_Product_Demand.csv
-rw-rw---- 1 wsuser wscommon 78K Nov  9 20:54 Next_Three_Month_Forecasts.csv


In [12]:
df_data_2 = pd.read_csv("Next_Three_Month_Forecasts.csv")

#### Pull in the demand data
This dataset provides the forecast demand for every warehouse product combination available. 

In [13]:
# Some preprocessing on the Next_Three_Month_Forecasts.csv from above cell
df_data_2 = df_data_2.rename(columns={'Unnamed: 0': 'Date'})
df_data_2.head()

Date  Product_0001_W2  Product_0003_W2  Product_0004_W2  \
0  2017-01-01             1643             1838             2131   
1  2017-02-01             2030              985             1609   
2  2017-03-01              127             2897             2293   

   Product_0005_W2  Product_0006_W2  Product_0007_W2  Product_0008_W2  \
0                0                2             6306             5721   
1               78              380             5473            17136   
2             1549              610             4798             8784   

   Product_0009_W2  Product_0010_W2  ...  Product_2138_W1  Product_2142_W1  \
0             6295             1701  ...               52                1   
1             2674              906  ...              469                9   
2             4364             1796  ...              249                0   

   Product_2144_W1  Product_2147_W1  Product_2164_W1  Product_2165_W1  \
0               57                3                1                5   
1              320                2                6                4   
2              179                2               13               16   

   Product_2166_W1  Product_2167_W1  Product_2168_W1  Product_2169_W1  
0               26              146                0               13  
1               40              192               17                8  
2               13              311               59               19  

[3 rows x 2850 columns]

In [14]:
cols = df_data_2.columns[1:]
new_df_1 = df_data_1[df_data_1['Product_Code_Warehouse'].isin(list(cols))]
new_df_2 = pd.DataFrame()
new_df_2["Product_Code_Warehouse"] = df_data_2.columns[1:]
new_df_2["Forecast"] = df_data_2.iloc[2][1:].values


main_df = pd.merge(new_df_1.drop("Forecast",axis=1),new_df_2, on="Product_Code_Warehouse", how="right")
main_df.dropna(subset=["Product_Code"], inplace=True)
main_df.dropna(subset=["capacity"], inplace=True)
main_df["Forecast"] = main_df["Forecast"].astype(int)
main_df["Time_Enroute"] = main_df["Time_Enroute"].astype(int)

In [15]:
demand_df = main_df[['Destination','Product_Code','Forecast']].drop_duplicates().reset_index(drop = True)

#### Pull in our backlog data (currently inventory)
This csv file contains the backlog for every product in every warehouse.

In [16]:
!wget https://raw.githubusercontent.com/IBMDataScience/sample-notebooks/master/Files/Data/Backlog.csv
!ls -lh

--2021-11-09 20:54:53--  https://raw.githubusercontent.com/IBMDataScience/sample-notebooks/master/Files/Data/Backlog.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56478 (55K) [text/plain]
Saving to: ‘Backlog.csv’

Backlog.csv         100%[===================>]  55.15K  --.-KB/s    in 0.001s  

2021-11-09 20:54:54 (54.7 MB/s) - ‘Backlog.csv’ saved [56478/56478]

total 133M
-rw-rw---- 1 wsuser wscommon 84M Nov  9 20:54 All_Routes_Table.csv
-rw-rw---- 1 wsuser wscommon 56K Nov  9 20:54 Backlog.csv
-rw-rw---- 1 wsuser wscommon 49M Nov  9 20:53 Historical_Product_Demand.csv
-rw-rw---- 1 wsuser wscommon 78K Nov  9 20:54 Next_Three_Month_Forecasts.csv


In [17]:
df_data_3 = pd.read_csv("Backlog.csv")

In [18]:
df_data_3.head()

Destination  Product_Code  backlog
0          W2  Product_0001      7.0
1          W2  Product_0003    229.0
2          W2  Product_0004     93.0
3          W2  Product_0005     63.0
4          W2  Product_0006     29.0

In [19]:
backlog_demand = df_data_3.merge(demand_df, left_on = ('Destination', 'Product_Code'), right_on = ('Destination', 'Product_Code'))
backlog_demand.head()

Destination  Product_Code  backlog  Forecast
0          W2  Product_0001      7.0       127
1          W2  Product_0003    229.0      2897
2          W2  Product_0004     93.0      2293
3          W2  Product_0005     63.0      1549
4          W2  Product_0006     29.0       610

# Create decision variables<a class="anchor" id="decision_variables"></a>
#### Re-usable functions to emphasize best-practice pattern
#### Added as new method on a CPLEX Model so usage is very close to a call to the other methods of Model

In [20]:
from docplex.mp.model import Model

def integer_var_series(self, df, **kargs):
    '''Create a Series of integer dvar for each row in the DF. Most effective method. Best practice.
    Result can be assigned to a column of the df.
    Usage:
        df['xDVar'] = mdl.integer_var_series(df, name = 'xDVar')
    Args:
        mdl (docplex.mp.model): CPLEX Model
        df (DataFrame): dataframe
        **kargs: arguments passed to mdl.integer_var_list method. E.g. 'name'
    Returns:
        (pandas.Series) with integer dvars, index matches index of df
    '''
    # We are re-using the index from the DF index:
    return pd.Series(self.integer_var_list(df.index, **kargs), index = df.index)

def continuous_var_series(self, df, **kargs):
    return pd.Series(self.continuous_var_list(df.index, **kargs), index = df.index)

def binary_var_series(self, df, **kargs):
    return pd.Series(self.binary_var_list(df.index, **kargs), index = df.index)

# Bind method to class:
Model.integer_var_series = integer_var_series
Model.continuous_var_series = continuous_var_series
Model.binary_var_series = binary_var_series

### Data Checker<a class="anchor" id="data_checker"></a>
For now assume that input data originates from a relational database that ensures:
* All the required columns are present
* The names of the columns are as expected
* Each column has the expected data type
* Null vs None-null values as expected
* Relational integrity: foreign keys exist as rows in their source table

### Pre-processing<a class="anchor" id="pre_procesing"></a>
The basic building blocks for this optimization problem are the `ProductionActivities` and `AggregateDemand`, i.e. what Product to produce on what Line:
* (key) Product
* (key) Line
* quantity

Steps:
1. Get active Demand and active Products
2. Create ProductionActivities that can manufacture a Product on a Production Line in a TimePeriod
3. Aggregate Demand

### ProductionActivities 
1. Create all combinations between a Product and Product Line.
2. Create all combinations between a Product, Warehouse and Production Line.

Production Activities are the basis for the production quantity decision variables.

### AggregateDemand
AggregateDemand is the basis for the demand fulfillment constraints and the backlog decision variables.<br>
We aggregate demand over each product, so we can define a backlog per product.<br>
In this simplified demo, we do not yet model backlog per time-period, which requires a backlog-resupply.

# Objective Function Definition<a class="anchor" id="objective"></a>

In [21]:

from IPython.display import clear_output
# Create production activities and aggregate demand tables for clarity
pa = main_df
ad = backlog_demand
backlog_penalty = 10 # For now, hard-code the back log penalty
pa.Miles = pa.Miles / 100000 # For now, hard-code cost. Drive down milage to create simplified cost per product

name_f = tuple(list(pa.prodx))
pa['xProd_vecm'] = mdl.integer_var_series(pa, lb = 0, name = name_f)
clear_output()

#### Minimize cost:
-  Production cost:
    -  _sum(pa) xProd[pa] * pa.costPerPart_
    -  i.e. the production quantity * the production cost per part
-  Backlog cost: 
    -  _sum(ad) xBacklog[ad] * backlogPenalty_
    -  i.e. the backlog quantity * backlog penalty. For now the backlog penalty is a static number (100)
    
The backlog costs are set an order of magnitude higher than the production cost. This will force the model to make backlog reduction the highest priority, before minimizing the production cost.<br>
_Note: an alternative approach is to define a lexicographical/hierarchical optimization approach._

#### Production Cost: 
The production cost is the amount of units supplied per given product route multiplied by the production cost.

$$
productionCost = \sum\limits_{\forall ProductionActivity} xProd[pa] * pa.costPerPart
$$

In [22]:
production_cost = mdl.sum(pa.xProd_vecm * pa.cost) #Uses vector multiplication, very efficient
mdl.add_kpi(production_cost, "Production Cost")

DecisionKPI(name=Production Cost,expr=0.020Product_0001S1M1W1W3W4W2+0.020Product_0001S1M1W1W4W3W2+0.02..)

#### Backlog Cost: 
Backlog cost is the number of products that can't immediately be created due to production capacity constraints for each warehouse multiplied by the penalty assigned for each backlog unit. 
$$
backlogCost = \sum\limits_{\forall AggregatedDemand} xBacklog[ad] * penalty
$$

In [23]:
ad['xBacklog'] = ad['Destination'] + ad['Product_Code']
backlog_f = tuple(list(ad.xBacklog))
ad['xBacklog_vecm'] = mdl.integer_var_series(ad, lb = 0, name = backlog_f)

In [24]:
backlog_cost = mdl.sum(ad.xBacklog_vecm * backlog_penalty)
mdl.add_kpi(backlog_cost, "Backlog Cost")

DecisionKPI(name=Backlog Cost,expr=10W2Product_0001+10W2Product_0003+10W2Product_0004+10W2Product_0..)

#### Travel Cost: 
Travel cost is the number of units supplied for each unique product route multiplied by the distance (as measured in miles).
$$
TravelCost = \sum\limits_{\forall ProductionActivity} xProd[pa] * xMile
$$

In [25]:
travel_cost = mdl.sum(pa.xProd_vecm * pa.Miles)
mdl.add_kpi(travel_cost, "Travel Cost")

DecisionKPI(name=Travel Cost,expr=0.000Product_0001S1M1W1W3W4W2+0.000Product_0001S1M1W1W4W3W2+0.00..)

#### Objective Function:
In this use case, our objective function attempts to minimize the backlog, production and travel costs. The cell below takes the various costs that we have created above and defines our objective function. 

In [26]:
mdl.minimize(backlog_cost + production_cost + travel_cost)

# Constraint Definition<a class="anchor" id="constraint"></a>
-  Production capacity:
    -  _For each line l:_ 
        -  _sum(pa in l) xProd[pa] / lineCapacity[l,pa.p] <= 1_
    -  Each line has a different capacity to produce a product. Therefore, we compute an expression for the line utilization per pa. The total utilization must be below 100%
-  Fulfill demand:
    -  _For each aggregated demand product p:_ 
        -  _sum(pa in p) xProd[pa] >= p.aggregatedDemand - xBacklog[p]_
    -  The total production for a part needs to be larger than the total demand, minus the backlog

In [27]:
ad.head()

Destination  Product_Code  backlog  Forecast        xBacklog   xBacklog_vecm
0          W2  Product_0001      7.0       127  W2Product_0001  W2Product_0001
1          W2  Product_0003    229.0      2897  W2Product_0003  W2Product_0003
2          W2  Product_0004     93.0      2293  W2Product_0004  W2Product_0004
3          W2  Product_0005     63.0      1549  W2Product_0005  W2Product_0005
4          W2  Product_0006     29.0       610  W2Product_0006  W2Product_0006

In [28]:
# create forecast compensating for backlog
ad['forecast_backlog'] = ad['Forecast'] - ad['backlog']

#### Demand Fulfillment Constraint: 
This constraint stipulates that for every product and its final warehouse destination the quantity supplied is always greater than or equal to the forecast minus the original backlog as well as any additional backlog added. 

In [29]:
# Demand Fulfillment Constraint
for (product, warehouse), group in pa.groupby(['Product_Code', 'Destination']):
    ad1 = ad[(ad['Product_Code'] == product) & (ad['Destination'] == warehouse)].sum()
    mdl.add_constraint(
        mdl.sum(group.xProd_vecm) 
         >= 
        ad1.forecast_backlog - ad1.xBacklog_vecm, 
         'DemandFulfillmentC_{}'.format(product)
    )

#### Capacity Constraint: 
This constraint stipulates that the amount of quantity supplied for every product supplier combination 
does not exceed the amount that the given supplier can create for a given product. 


In [30]:
# Capacity Constraint
for (product, supplier), group in pa.groupby(['Product_Code', 'SupplierID']):
    
    mdl.add_constraint(
        (mdl.sum(mdl.sum(group.xProd_vecm * (1/group.capacity)) <= 1)), 'WarehouseCapacityC_{}_{}'.format(product, supplier))

#### Define timeout: 
Optimization model does not endlessly try to find an optimal solution.

In [31]:
mdl.parameters.timelimit = TIME_LIMIT
print(mdl.parameters.print_info_to_string(print_defaults = False)) #Logs all parameter values that are different from the default

parameters.timelimit = 30.00000000000000



# Run CPLEX Model<a class="anchor" id="run_model"></a>
#### Now that our model is defined with an objective function and constraints, we can solve!

In [32]:
# CPLEX:
# Additionally you can add a time parameter to the solve method below: cplex_parameters = {'timelimit':TIME_LIMIT})
msol = mdl.solve(log_output = True)
if msol is not None:
    mdl.report()

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 2
CPXPARAM_TimeLimit                               30
Tried aggregator 3 times.
MIP Presolve eliminated 7349 rows and 436568 columns.
MIP Presolve modified 3454 coefficients.
Aggregator did 299 substitutions.
Reduced MIP has 2213 rows, 6723 columns, and 8576 nonzeros.
Reduced MIP has 303 binaries, 6420 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.67 sec. (435.75 ticks)
Found incumbent of value 6.9227447e+08 after 0.71 sec. (464.34 ticks)
Probing time = 0.00 sec. (0.11 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 824 rows and 4157 columns.
Aggregator did 3 substitutions.
Reduced MIP has 1386 rows, 2563 columns, and 4405 nonzeros.
Reduced MIP has 127 binaries, 2436 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (34.92 ticks)
Probing time = 0.00 sec. (0.06 ticks)
Tried aggregator 1 time.
MIP Presolve e

#### Conflict resolution<a class="anchor" id="conflict_resolution"></a>
For larger sized decision models, it can be hard to find what is causing the model solve to fail. This is where the CPLEX conflict refiner can help. If the model is infeasible, the conflict refiner will search for the minimal conflict. The minimal conflict is the smallest set of constraints and variables that are in conflict with each other, i.e. where it is impossible to find a feasible solution among the decision variables that satisfy all constraints in the set.

Knowing the conflict can help understanding the modelling error and thereby help debugging the model generation code.

For details, see the reference manual: http://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.conflict_refiner.html

Run the CPLEX conflict resolution to find out what constraints are conflicting:

In [33]:
from docplex.mp.conflict_refiner import ConflictRefiner
if msol is None:
    crefiner = ConflictRefiner() # Create an instance of the ConflictRefiner
    conflicts = crefiner.refine_conflict(mdl) # Run the conflict refiner
    for c in conflicts:
        print(c.element) # Display conflict result in a little more compact format than ConflictRefiner.display_conflicts

# Post Processing<a class="anchor" id="post"></a>

#### The cell below provides the total production, backlog, and travel costs - respectively

In [34]:
all_kpis = [(kp.name, kp.compute()) for kp in mdl.iter_kpis()]
df_kpis = pd.DataFrame(all_kpis, columns = ['kpi', 'value'])
df_kpis

kpi         value
0  Production Cost  4.819807e+06
1     Backlog Cost  1.047429e+08
2      Travel Cost  6.578811e+00

#### Extract solution
-  ProductionActivity
-  Backlog

*Note: The reason we explicitly extract the solution in a separate column, versus indirectly referring to the value using the `.solution_value` attribute of the dvar is that the column value will allow us to do vector operations to compute derived values.*

In [35]:
pa['production_quantity'] = [dvar.solution_value for dvar in pa.xProd_vecm]
ad['backlog_quantity'] = [dvar.solution_value for dvar in ad.xBacklog_vecm]
pa_nozeros = pa[pa['production_quantity'] > 0]

In [36]:
!wget https://raw.githubusercontent.com/IBMDataScience/sample-notebooks/master/Files/Data/Product_Category_Legend.csv
!ls -lh

--2021-11-09 20:55:21--  https://raw.githubusercontent.com/IBMDataScience/sample-notebooks/master/Files/Data/Product_Category_Legend.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86509 (84K) [text/plain]
Saving to: ‘Product_Category_Legend.csv’

Product_Category_Le 100%[===================>]  84.48K  --.-KB/s    in 0.002s  

2021-11-09 20:55:22 (54.6 MB/s) - ‘Product_Category_Legend.csv’ saved [86509/86509]

total 133M
-rw-rw---- 1 wsuser wscommon 84M Nov  9 20:54 All_Routes_Table.csv
-rw-rw---- 1 wsuser wscommon 56K Nov  9 20:54 Backlog.csv
-rw-rw---- 1 wsuser wscommon 49M Nov  9 20:53 Historical_Product_Demand.csv
-rw-rw---- 1 wsuser wscommon 78K Nov  9 20:54 Next_Three_Month_Forecasts.csv
-rw-rw---- 1 wsuser wscommon 85K Nov  9 20:55 Product_Categor

In [37]:
df_data_4 = pd.read_csv("Product_Category_Legend.csv")

In [38]:
df_data_4.head()

Unnamed: 0 Product_Category  Product_Code  Unnamed: 0.1
0           0     Category_001  Product_0256     134466969
1           1     Category_001  Product_0257      67350671
2           2     Category_001  Product_0258      85880185
3           3     Category_001  Product_0259      54751149
4           4     Category_001  Product_0260     100110956

In [39]:
df_data_4.columns = ['Index', 'Product_Category', 'Product_Code', 'Total']
del df_data_4['Index']
del df_data_4['Total']

In [40]:
!wget https://raw.githubusercontent.com/IBMDataScience/sample-notebooks/master/Files/Data/Locations.csv
!ls -lh
df_data_5 = pd.read_csv("Locations.csv")
df_data_5.head()

--2021-11-09 20:55:24--  https://raw.githubusercontent.com/IBMDataScience/sample-notebooks/master/Files/Data/Locations.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 806 [text/plain]
Saving to: ‘Locations.csv’

Locations.csv       100%[===================>]     806  --.-KB/s    in 0s      

2021-11-09 20:55:24 (28.6 MB/s) - ‘Locations.csv’ saved [806/806]

total 133M
-rw-rw---- 1 wsuser wscommon 84M Nov  9 20:54 All_Routes_Table.csv
-rw-rw---- 1 wsuser wscommon 56K Nov  9 20:54 Backlog.csv
-rw-rw---- 1 wsuser wscommon 49M Nov  9 20:53 Historical_Product_Demand.csv
-rw-rw---- 1 wsuser wscommon 806 Nov  9 20:55 Locations.csv
-rw-rw---- 1 wsuser wscommon 78K Nov  9 20:54 Next_Three_Month_Forecasts.csv
-rw-rw---- 1 wsuser wscommon 85K Nov  9 20:55 Product_C

WarehouseID    Warehouse Warehouse_State   Warehouse_Region       Type  \
0          W1       Eugene              OR  West Coast Region  Warehouse   
1          W2  Los Angeles              CA  West Coast Region  Warehouse   
2          W3   Sacramento              CA  West Coast Region  Warehouse   
3          W4      Seattle              WA  West Coast Region  Warehouse   
4          S1     Portland              OR  West Coast Region   Supplier   

   Color  Address      Lat      Long  
0   Blue      NaN  44.0900 -123.0600  
1   Blue      NaN  34.0522 -118.2437  
2   Blue      NaN  38.9100 -121.4200  
3   Blue      NaN  47.7000 -122.3500  
4  White      NaN  45.4600 -122.6100

In [41]:
df_data_5 = df_data_5.rename(columns = {'Warehouse':'Warehouse_City'})

In [42]:
pa_withproduct = pa_nozeros.merge(df_data_4, how = 'left',left_on = 'Product_Code',right_on = 'Product_Code')
pa_prodandlocation = pa_withproduct.merge(df_data_5, how = 'left', left_on = 'Destination', right_on = 'WarehouseID')

#### Save the solution and output to our project

In [ ]:
project.save_data('Production_and_Location.csv', pa_prodandlocation.to_csv(index=False), overwrite=True)

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020 <br>
All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.**